# PSO

In [1]:
from pyswarm import pso
import matplotlib.pyplot as plt

In [2]:
def fitfunc(x):
    return -(9.24 + 0.66*x[0] - 1.82*x[1] + 0.060*x[2] + 0.20*x[0]*x[1] + 
             0.0033*x[0]*x[2] + 0.013*x[1]*x[2] - 0.073*x[0]**2 + 0.11*x[1]**2 + 0.0015*x[2]**2)

lb = [1, 2, 0]
ub = [3, 6, 15]

x, y = pso(fitfunc, lb, ub, swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=100, minstep=1e-8, 
           minfunc=1e-8, debug=False);

print(x,y)

Stopping search: maximum iterations reached --> 100
[ 3.  2. 15.] -10.339


# GA

In [16]:
from numpy.random import randint
from numpy.random import rand
 
# objective function
def objective(x):
	return -(9.24 + 0.66*x[0] - 1.82*x[1] + 0.060*x[2] + 0.20*x[0]*x[1] + 0.0033*x[0]*x[2] + 0.013*x[1]*x[2] - 0.073*x[0]**2 + 0.11*x[1]**2 + 0.0015*x[2]**2)
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded
 
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]
 
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]
 
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]
 
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]
 
# define range for input
bounds = [[1,3], [2,6], [0,15]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 32
# define the population size
n_pop = 40
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([1.1057034828700125, 3.137605261988938, 14.42999088903889]) = -7.766201
>0, new best f([2.9858422675170004, 5.416616041213274, 4.698855666210875]) = -7.855828
>0, new best f([1.3322648601606488, 2.090725500136614, 8.865293000126258]) = -8.152247
>0, new best f([2.924963979050517, 3.1674516452476382, 12.789822133490816]) = -9.400564
>1, new best f([2.9911259566433728, 4.3840456251055, 14.268172255251557]) = -9.434371
>1, new best f([2.968692956492305, 4.999572616070509, 14.487049434101209]) = -9.442263
>1, new best f([2.924984857439995, 3.143359161913395, 13.844982533482835]) = -9.568375
>2, new best f([2.924963975325227, 3.167466649785638, 14.268175831530243]) = -9.624405
>2, new best f([2.924984857439995, 3.1435411227867007, 14.66481855721213]) = -9.693976
>3, new best f([2.968708712141961, 2.557546403259039, 14.487020376836881]) = -9.946908
>3, new best f([2.924984857439995, 2.174609161913395, 13.844980469439179]) = -10.023727
>4, new best f([2.924984857439995, 2.17473